In [39]:

import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder


In [40]:
data = pd.read_csv('george_wassouf_lyrics_dataset.csv')
data.head()

,Track Title,Track Lyrics,Release year,Composer,Lyricist
0,Aah Habaieb - آه حبايب,اه حبايب بس فين الحب غايب\nوليه يا قلبي ما لك...,1995,صلاح الشرنوبي,وليد رزيقه
1,Aalem Albi El Shok - علم قلبي الشوق,\nيا ليلي يا ليلي يا ليل\n\nعلم قلبي الشوق الش...,2008,صلاح الشرنوبي,احمد شتا
2,Addi Eli Kanou - أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا\nواللي ما ...,2008,امجد العاطفي,احمد سامي
3,Alb Al Asheq Daleiloh - قلب العاشق دليله,قلب العاشق دليله\nقلب العاشق دليله\n\nقلب العا...,2002,امجد العاطفي,عوض بدوي
4,Al Forqah Saabah - الفرقة صعبة,مش قادر اصبرعغيابا\nكل ليله بجمع دموعي\nو بسهر...,2009,جورج يزبك,توفيق بركات


In [41]:
# Number of songs
no_of_songs = data.shape[0]
print(no_of_songs)
# Number of attributes
no_of_attributes = data.shape[1]
print(no_of_attributes)
# Attribute names
attrs = data.columns.tolist()
print(attrs)

78
5
['Track Title', 'Track Lyrics', 'Release year', 'Composer', 'Lyricist']


In [42]:
track_lyrics_df = data['Track Lyrics'].to_frame()
print(track_lyrics_df.head())

                                        Track Lyrics
0   اه حبايب بس فين الحب غايب\nوليه يا قلبي ما لك...
1  \nيا ليلي يا ليلي يا ليل\n\nعلم قلبي الشوق الش...
2   أدي اللي كانوا يا هوا أقرب ما لينا\nواللي ما ...
3  قلب العاشق دليله\nقلب العاشق دليله\n\nقلب العا...
4  مش قادر اصبرعغيابا\nكل ليله بجمع دموعي\nو بسهر...


In [43]:
data['Track Lyrics'] = data['Track Lyrics'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data.head()

,Track Title,Track Lyrics,Release year,Composer,Lyricist
0,Aah Habaieb - آه حبايب,اه حبايب بس فين الحب غايب\nوليه يا قلبي ما لك...,1995,صلاح الشرنوبي,وليد رزيقه
1,Aalem Albi El Shok - علم قلبي الشوق,\nيا ليلي يا ليلي يا ليل\n\nعلم قلبي الشوق الش...,2008,صلاح الشرنوبي,احمد شتا
2,Addi Eli Kanou - أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا\nواللي ما ...,2008,امجد العاطفي,احمد سامي
3,Alb Al Asheq Daleiloh - قلب العاشق دليله,قلب العاشق دليله\nقلب العاشق دليله\n\nقلب العا...,2002,امجد العاطفي,عوض بدوي
4,Al Forqah Saabah - الفرقة صعبة,مش قادر اصبرعغيابا\nكل ليله بجمع دموعي\nو بسهر...,2009,جورج يزبك,توفيق بركات


In [44]:
data['Track Title'] = data['Track Title'].apply(lambda x: re.sub(r'[-A-Za-z]', '', x))
data.head()

,Track Title,Track Lyrics,Release year,Composer,Lyricist
0,آه حبايب,اه حبايب بس فين الحب غايب\nوليه يا قلبي ما لك...,1995,صلاح الشرنوبي,وليد رزيقه
1,علم قلبي الشوق,\nيا ليلي يا ليلي يا ليل\n\nعلم قلبي الشوق الش...,2008,صلاح الشرنوبي,احمد شتا
2,أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا\nواللي ما ...,2008,امجد العاطفي,احمد سامي
3,قلب العاشق دليله,قلب العاشق دليله\nقلب العاشق دليله\n\nقلب العا...,2002,امجد العاطفي,عوض بدوي
4,الفرقة صعبة,مش قادر اصبرعغيابا\nكل ليله بجمع دموعي\nو بسهر...,2009,جورج يزبك,توفيق بركات


In [45]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

def remove_newline_chars(text):
    return text.replace('\n', ' ').replace('\r', '')

def remove_diacritics(text):
    arabic_diacritics = re.compile("""
                                ّ    | # Tashdid
                               ~    | # Shada 
                                َ    | # Fatha
                                ً    | # Tanwin Fath
                                ُ    | # Damma
                                ٌ    | # Tanwin Damm
                                ِ    | # Kasra
                                ٍ    | # Tanwin Kasr
                                ْ    | # Sukun
                                ـ     # Tatwil/Kashida
                            """, re.VERBOSE)
    return re.sub(arabic_diacritics, '', text)

data['Track Lyrics'] = data['Track Lyrics'].apply(remove_punctuation)
data['Track Lyrics'] = data['Track Lyrics'].apply(remove_newline_chars)
data['Track Lyrics'] = data['Track Lyrics'].apply(remove_diacritics)
data['Track Title'] = data['Track Title'].apply(remove_punctuation)
data['Track Title'] = data['Track Title'].apply(remove_newline_chars)
data['Track Title'] = data['Track Title'].apply(remove_diacritics)

data.head()

,Track Title,Track Lyrics,Release year,Composer,Lyricist
0,آه حبايب,اه حبايب بس فين الحب غايب وليه يا قلبي ما لكش...,1995,صلاح الشرنوبي,وليد رزيقه
1,علم قلبي الشوق,يا ليلي يا ليلي يا ليل علم قلبي الشوق الشوق ...,2008,صلاح الشرنوبي,احمد شتا
2,أدي اللي كانوا,أدي اللي كانوا يا هوا أقرب ما لينا واللي ما ه...,2008,امجد العاطفي,احمد سامي
3,قلب العاشق دليله,قلب العاشق دليله قلب العاشق دليله قلب العاشق ...,2002,امجد العاطفي,عوض بدوي
4,الفرقة صعبة,مش قادر اصبرعغيابا كل ليله بجمع دموعي و بسهر م...,2009,جورج يزبك,توفيق بركات


In [46]:
def standardizing_arabic_text(text):
    text = re.sub(r'ي\b', 'ى', text)
    text = text.replace('أ', 'ا')
    text = text.replace('آ', 'ا')
    text = re.sub(r'\bإ', 'ا', text)
    text = re.sub(r'ة\b', 'ه', text)
    return text

data['Track Lyrics'] = data['Track Lyrics'].apply(standardizing_arabic_text)
data['Track Title'] = data['Track Title'].apply(standardizing_arabic_text)
data.head()

,Track Title,Track Lyrics,Release year,Composer,Lyricist
0,اه حبايب,اه حبايب بس فين الحب غايب وليه يا قلبى ما لكش...,1995,صلاح الشرنوبي,وليد رزيقه
1,علم قلبى الشوق,يا ليلى يا ليلى يا ليل علم قلبى الشوق الشوق ...,2008,صلاح الشرنوبي,احمد شتا
2,ادى اللى كانوا,ادى اللى كانوا يا هوا اقرب ما لينا واللى ما ه...,2008,امجد العاطفي,احمد سامي
3,قلب العاشق دليله,قلب العاشق دليله قلب العاشق دليله قلب العاشق ...,2002,امجد العاطفي,عوض بدوي
4,الفرقه صعبه,مش قادر اصبرعغيابا كل ليله بجمع دموعى و بسهر م...,2009,جورج يزبك,توفيق بركات


In [47]:
total_words_before = data['Track Lyrics'].apply(lambda x: len(x.split())).sum()
print(f'Total number of words before removing stopwords: {total_words_before}')

Total number of words before removing stopwords: 31921


In [48]:
words_before = data['Track Lyrics'].apply(lambda x: x.split()).explode()

In [49]:
stop_words = set(stopwords.words('arabic'))
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

data['Track Lyrics'] = data['Track Lyrics'].apply(remove_stopwords)
data['Track Title'] = data['Track Title'].apply(remove_stopwords)
data.head()

,Track Title,Track Lyrics,Release year,Composer,Lyricist
0,اه حبايب,اه حبايب فين الحب غايب وليه قلبى لكش نايب شافو...,1995,صلاح الشرنوبي,وليد رزيقه
1,قلبى الشوق,ليلى ليلى ليل قلبى الشوق الشوق عينى السهر خدنى...,2008,صلاح الشرنوبي,احمد شتا
2,ادى اللى كانوا,ادى اللى كانوا هوا اقرب لينا واللى هانوا هوا ف...,2008,امجد العاطفي,احمد سامي
3,قلب العاشق دليله,قلب العاشق دليله قلب العاشق دليله قلب العاشق د...,2002,امجد العاطفي,عوض بدوي
4,الفرقه صعبه,مش قادر اصبرعغيابا ليله بجمع دموعى بسهر دمعات ...,2009,جورج يزبك,توفيق بركات


In [50]:
total_words_after = data['Track Lyrics'].apply(lambda x: len(x.split())).sum()
print(f'Total number of words after removing stopwords: {total_words_after}')

Total number of words after removing stopwords: 26588


In [51]:
words_after = data['Track Lyrics'].apply(lambda x: x.split()).explode()

In [52]:
eliminated_words = words_before[~words_before.isin(words_after)]
print(f'Total number of eliminated words: {len(eliminated_words)}')
print(f'Eliminated words: {eliminated_words.tolist()}')

Total number of eliminated words: 5333
Eliminated words: ['بس', 'يا', 'ما', 'بس', 'يا', 'يا', 'ما', 'ما', 'ما', 'بس', 'يا', 'يا', 'يا', 'كان', 'حبيب', 'بس', 'بس', 'يا', 'ما', 'بس', 'يا', 'يا', 'ما', 'ما', 'ما', 'بس', 'يا', 'يا', 'يا', 'كان', 'حبيب', 'بس', 'يا', 'يا', 'يا', 'علم', 'علم', 'ع', 'علم', 'علم', 'ع', 'ما', 'علم', 'علم', 'ع', 'يا', 'يا', 'يا', 'يا', 'ما', 'علم', 'علم', 'ع', 'يا', 'قد', 'فوق', 'ما', 'ما', 'قد', 'فوق', 'ما', 'ما', 'ما', 'علم', 'علم', 'ع', 'يا', 'يا', 'يا', 'علم', 'علم', 'ع', 'علم', 'علم', 'ع', 'ما', 'علم', 'علم', 'ع', 'يا', 'يا', 'يا', 'يا', 'ما', 'علم', 'علم', 'ع', 'يا', 'قد', 'فوق', 'ما', 'ما', 'قد', 'فوق', 'ما', 'ما', 'ما', 'علم', 'علم', 'ع', 'يا', 'ما', 'ما', 'يا', 'يا', 'ما', 'ما', 'يا', 'يا', 'يا', 'يا', 'ما', 'ما', 'يا', 'قبل', 'ما', 'على', 'كل', 'قبل', 'ما', 'على', 'كل', 'يا', 'ما', 'ما', 'ما', 'يا', 'يا', 'من', 'غير', 'كان', 'على', 'يا', 'من', 'غير', 'كان', 'على', 'يا', 'يا', 'ما', 'ما', 'ما', 'يا', 'يا', 'ما', 'ما', 'يا', 'يا', 'يا', 'ما', 'ما', 'يا', 

In [53]:
# Tokenize and join with newline characters
data['Tokenized & Preprocessed Lyrics'] = data['Track Lyrics'].apply(lambda x: x.split())
data['Tokenized & Preprocessed Lyrics'] = data['Tokenized & Preprocessed Lyrics'].apply(lambda x: '\n'.join(x))

# Display the result
print(data[['Track Title', 'Track Lyrics', 'Tokenized & Preprocessed Lyrics']].head())

# Save with UTF-8-SIG encoding
data.to_csv('george_wassouf_lyrics_tokenized.csv', index=False, encoding='utf-8-sig')

        Track Title                                       Track Lyrics  \
0          اه حبايب  اه حبايب فين الحب غايب وليه قلبى لكش نايب شافو...   
1        قلبى الشوق  ليلى ليلى ليل قلبى الشوق الشوق عينى السهر خدنى...   
2    ادى اللى كانوا  ادى اللى كانوا هوا اقرب لينا واللى هانوا هوا ف...   
3  قلب العاشق دليله  قلب العاشق دليله قلب العاشق دليله قلب العاشق د...   
4       الفرقه صعبه  مش قادر اصبرعغيابا ليله بجمع دموعى بسهر دمعات ...   

                     Tokenized & Preprocessed Lyrics  
0  اه\nحبايب\nفين\nالحب\nغايب\nوليه\nقلبى\nلكش\nن...  
1  ليلى\nليلى\nليل\nقلبى\nالشوق\nالشوق\nعينى\nالس...  
2  ادى\nاللى\nكانوا\nهوا\nاقرب\nلينا\nواللى\nهانو...  
3  قلب\nالعاشق\nدليله\nقلب\nالعاشق\nدليله\nقلب\nا...  
4  مش\nقادر\nاصبرعغيابا\nليله\nبجمع\nدموعى\nبسهر\...  


In [54]:
print("Total number of songs in the dataset:", no_of_songs)
print("Total number of composers:", data['Composer'].nunique())
print("Total number of lyricists:", data['Lyricist'].nunique())

Total number of songs in the dataset: 78
Total number of composers: 17
Total number of lyricists: 34


Initial invistigation for keywords

In [60]:

# Define the keyword
keyword = "طبيب"
filtered_songs = data[data['Tokenized & Preprocessed Lyrics'].str.contains(keyword, na=False, regex=True)]

# Display the result
print(filtered_songs['Track Title'])


21    الزمن دوار
44          عاشق
71     طبيب جراح
Name: Track Title, dtype: object


In [64]:
# Define the keyword
keyword = "حب"

# Filter songs containing the keyword
filtered_songs = data[data['Tokenized & Preprocessed Lyrics'].str.contains(keyword, na=False, regex=True)].copy()

# Add a column to count occurrences of the keyword in each song
filtered_songs.loc[:, 'Keyword Occurrences'] = filtered_songs['Tokenized & Preprocessed Lyrics'].str.count(keyword)

# Display the result
print(filtered_songs[['Track Title', 'Keyword Occurrences']])

         Track Title  Keyword Occurrences
0           اه حبايب                   24
1         قلبى الشوق                    8
2     ادى اللى كانوا                   12
3   قلب العاشق دليله                   10
4        الفرقه صعبه                   12
..               ...                  ...
73        ياه عالزمن                    8
74        نجوم الليل                   16
75        يوم الوادع                   24
76       زمن العجايب                   18
77            ذكريات                   12

[66 rows x 2 columns]
